In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd "gdrive/My Drive/EmotionRecognition"

In [ ]:
! git clone https://github.com/BPTI-LT/Emociju-duomenu-baze.git

In [ ]:
! zip -r EMOdb.zip EMOdb

In [ ]:
cp '/content/gdrive/MyDrive/EmotionRecognition/EMOdb.zip' '/content/EMOdb.zip'

In [ ]:
%cd "/content"

In [ ]:
! unzip 'EMOdb.zip'

In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio

In [ ]:
paths = [os.path.join(d, x)
            for d, dirs, files in os.walk('EMOdb')
            for x in files if x.endswith(".wav")]

paths.sort()

print(paths)

In [ ]:
class_names = ['Baime', 'Dziaugsmas', 'Liudesys', 'Neutrali', 'Nuostaba', 'Pasislykstejimas', 'Pyktis']
labels_names = [path.split('/')[2].split(' ')[1] for path in paths]

labels = [class_names.index(label_name)
          for label_name in labels_names]

print (labels_names)

In [ ]:
Data = []
iterations = 0

for path in paths:
  audio_data, sampling_rate = librosa.load(path)
  #Data.append(audio_data)
    
  mfccs = np.mean(librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=13), axis=0)
  #mfccs = np.array(librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=13), axis=0)
  #mfccs = librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=13)
  Data.append(mfccs)

  if (iterations % 10 == 0):
    print(iterations, ' | ', len(mfccs))
  
  iterations += 1
  if (iterations >= 1500):
      break;

print(len(paths))

In [ ]:
mean = int(np.mean([len(i) for i in Data]))
print(mean)

In [ ]:
Data2D=np.zeros([iterations,mean])

for i,s in enumerate(Data):
    Data2D[i,:len(s)]=s[:mean]
    
X = np.expand_dims(Data2D, axis=2)
X.shape

In [ ]:
N=Data2D.shape[0]

Y=np.array(labels[:N])
Y.shape

In [ ]:
p=np.random.permutation(N)

X=X[p]
Y=Y[p]  #sumaisymas

x_train=X[:int(0.7*N)]
y_train=Y[:int(0.7*N)]

x_val=X[int(0.7*N):]
y_val=Y[int(0.7*N):]

In [ ]:
import tensorflow as tf
import keras
L = mean

In [ ]:
input = tf.keras.Input(shape=(L, 1))
x=tf.keras.layers.Conv1D(128,5,activation='relu',padding='same')(input)  
x=tf.keras.layers.Conv1D(128,5,activation='relu',padding='same')(x)

x=tf.keras.layers.AveragePooling1D(2)(x)
x=tf.keras.layers.BatchNormalization()(x)

x=tf.keras.layers.Conv1D(128,5,activation='relu',padding='same')(x)
x=tf.keras.layers.Conv1D(128,5,activation='relu',padding='same')(x)

x=tf.keras.layers.AveragePooling1D(2)(x)
x=tf.keras.layers.BatchNormalization()(x)

x=tf.keras.layers.Conv1D(128,5,activation='relu',padding='same')(x)
x=tf.keras.layers.Conv1D(128,5,activation='relu',padding='same')(x)

x = tf.keras.layers.GlobalAveragePooling1D()(x)

x= tf.keras.layers.Dense(7,activation='softmax')(x)
model = tf.keras.Model(inputs=input,outputs=x)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5, min_lr=1e-9,verbose=1)
]

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['acc'])

In [ ]:
cnnhistory = model.fit(x_train,y_train,validation_data=(x_val,y_val),callbacks=callbacks,batch_size=32,shuffle=True,epochs=100)

In [ ]:
plt.plot(cnnhistory.history['acc'])
plt.plot(cnnhistory.history['val_acc'])
plt.title('Modelio tikslumas')
plt.ylabel('Tisklumas')
plt.xlabel('Epocha')
plt.legend(['Mokymosi', 'Įverinimo'], loc='upper left')
plt.show()

In [ ]:
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('Modelio nuostoliai')
plt.ylabel('Nuostolis')
plt.xlabel('Epocha')
plt.legend(['Mokymosi', 'Įverinimo'], loc='upper left')
plt.show()